In [1]:
import numpy as np
import pandas as pd
from shutil import copyfile
from astropy.cosmology import Planck13 as cosmo
import fsps
from matplotlib import gridspec
from astropy.io import fits
from astropy import wcs
from astropy.modeling.models import Sersic2D
from astropy.table import Table
from sim_engine import Scale_model
from spec_tools import Source_present, Oldest_galaxy, Sig_int, Smooth
from spec_stats import Smooth, Highest_density_region
from spec_id import *
from spec_stats import Highest_density_region, Linear_fit
from spec_exam import Gen_spec
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, interp2d
from glob import glob
import seaborn as sea
import os
from grizli import multifit
from grizli import model
from sim_engine import forward_model_grism
from astropy.wcs import wcs

import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
R = robjects.r
pandas2ri.activate()

from time import time
sea.set(style='white')
sea.set(style='ticks')
sea.set_style({'xtick.direct'
               'ion': 'in','xtick.top':True,'xtick.minor.visible': True,
               'ytick.direction': "in",'ytick.right': True,'ytick.minor.visible': True})
cmap = sea.cubehelix_palette(12, start=2, rot=.2, dark=0, light=1.0, as_cmap=True)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

### set home for files
hpath = os.environ['HOME'] + '/'

if hpath == '/Users/Vince.ec/':
    dpath = '/Volumes/Vince_research/Data/' 
    
else:
    dpath = hpath + 'Data/' 

In [2]:
buldge_cat = Table.read(fits.open('../dataframes/galaxy_frames/goods_bulge.fits'), format = 'fits').to_pandas()

In [5]:
SF_GSD = pd.read_pickle('../Casey_data/massMetal_GSD.pkl')
SF_GND = pd.read_pickle('../Casey_data/massMetal_GND.pkl')

Q_GSD = pd.read_pickle('../dataframes/galaxy_frames/GS_cnd.pkl')
Q_GND = pd.read_pickle('../dataframes/galaxy_frames/GN_cnd.pkl')

In [37]:
def find_cand_id(gal_ra, gal_dec, cat_ra, cat_dec, cat_id, galid):
    r = (1. / 60 / 60) * 0.05
    for i in range(len(gal_ra)):
        cosr = np.sin(np.radians(gal_dec[i])) * np.sin(np.radians(cat_dec)) + np.cos(np.radians(gal_dec[i]))\
                * np.cos(np.radians(cat_dec)) * np.cos(np.radians(gal_ra[i]) - np.radians(cat_ra))
        rad = np.arccos(cosr)
        rad = np.degrees(rad)
        for ii in range(len(rad)):
            if rad[ii] < r:
                galid[i] = cat_id[ii]
    return galid

In [42]:
galid = np.repeat(-99.0, len(buldge_cat.ra))

inouts = find_cand_id(buldge_cat.ra.values,  buldge_cat.dec.values, Q_GSD.ra.values, Q_GSD.dec.values, Q_GSD.id.values, galid)
inouts = find_cand_id(buldge_cat.ra.values,  buldge_cat.dec.values, Q_GND.ra.values, Q_GND.dec.values, Q_GND.id.values, galid)
inouts = find_cand_id(buldge_cat.ra.values,  buldge_cat.dec.values, SF_GSD.RA.values, SF_GSD.DEC.values, SF_GSD.ID.values, galid)
inouts = find_cand_id(buldge_cat.ra.values,  buldge_cat.dec.values, SF_GND.RA.values, SF_GND.DEC.values, SF_GND.ID.values, galid)


In [43]:
inouts

array([  -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,
         -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,
         -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,
         -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,
         -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,
         -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,
         -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,
         -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,
         -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,
         -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,
         -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,
         -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,
         -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,
         -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,   -99.,
      

In [36]:
buldge_cat.query('inouts > 0')[:5]

,idx,candels_id,ra,dec,field,Hmag,UV_col,z,z_type,z_phot,...,stack_flux_100um,stack_flux_160um,stack_cov_100um,stack_cov_160um,flux_100um,flux_160um,flux_250um,flux_350um,flux_500um,inouts
133,10390,10391,53.164993,-27.819336,1,19.610001,1.920000,0.980,1,0.972,...,0.207422,-0.076651,934.430909,1199.255412,-99.0,-99.0,-99.0,-99.0,-99.0,19148.0
156,12003,12004,53.137768,-27.802094,1,21.850000,1.900000,1.188,1,1.166,...,-0.165395,-0.043174,933.618689,1196.323600,-99.0,-99.0,-99.0,-99.0,-99.0,23421.0
168,13380,13381,53.149151,-27.792984,1,22.200001,1.700001,1.223,1,1.255,...,-0.126576,0.117696,939.937875,1185.859196,-99.0,-99.0,-99.0,-99.0,-99.0,25641.0
174,13786,13787,53.150833,-27.790607,1,22.610001,1.809999,1.439,0,1.439,...,0.130669,0.224460,938.922605,1196.571704,-99.0,-99.0,-99.0,-99.0,-99.0,26272.0
187,14838,14839,53.165161,-27.785866,1,21.180000,1.779999,1.317,1,1.221,...,0.138431,-0.102548,935.238401,1190.719659,-99.0,-99.0,-99.0,-99.0,-99.0,26915.0


In [25]:
Q_GSD

,dec,id,ra,zphot,X_AGN,SNR
0,-27.824564,18642,53.161738,0.837038,---,19.826768
1,-27.819326,19148,53.164983,0.972842,---,59.334098
2,-27.819042,19872,53.165850,1.079583,---,-0.958594
3,-27.802087,23421,53.137760,1.188682,AGN,13.274236
4,-27.798117,24148,53.186408,1.268042,---,-0.443343
5,-27.792980,25641,53.149147,1.186893,AGN,16.362752
6,-27.791271,25799,53.157186,1.116242,---,1.041127
7,-27.790772,26139,53.125080,1.587231,---,28.782653
8,-27.790601,26272,53.150826,1.563997,---,7.138911
9,-27.785866,26915,53.165159,1.320118,AGN,31.125360
